In [1]:
import sys
sys.path.append("..")
import logging
import random
import neptune
import numpy as np
from time import strftime
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils

CONFIG_PATH = '../configs/defaults.yml'


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ubuntu/.local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:
configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
random.seed(configs['runtime']['random_seed'])

# From command line
epochs_num = 1
rounds_num = 1
log_enable = False
output_dir = None
neptune_enable = False

fl = FederatedLearning(
    configs['runtime']['server_w0_batch_size'], 
    configs['runtime']['test_batch_size'], 
    configs['runtime']['lr'], 
    0.0,
    configs['runtime']['momentum'], 
    neptune_enable, log_enable, 
    configs['log']['interval'], 
    output_dir, 
    configs['runtime']['random_seed'])

fl.create_server()
fl.create_server_model()

raw_data = utils.preprocess_leaf_data(
    utils.load_leaf_train(configs['data']['FEMNIST_PATH']),
    min_num_samples=100,only_digits=True)


2020-11-12 08:20:42,439 Initializing Federated Learning class...
2020-11-12 08:20:43,226 Creating the server...
2020-11-12 08:20:43,228 Creating a model for the server...
2020-11-12 08:20:43,238 Loading train dataset from /home/ubuntu/data/leaf_non_iid/data/femnist/data
2020-11-12 08:20:43,239 Loading 1 out of 2 files...
2020-11-12 08:20:49,030 Loading 2 out of 2 files...
2020-11-12 08:20:55,273 Start processing of femnist data...


In [3]:


logging.info("Select data from only {} workers to be used...".format(configs['runtime']['femnist_total_users_num']))
workers_idx = list(raw_data.keys())[:configs['runtime']['femnist_total_users_num']]
logging.info(len(workers_idx))

server_train_images, server_train_labels = fl.create_server_femnist_dataset(
    raw_data, workers_idx, configs['runtime']['public_data_percentage'])
server_federated_train_dataloader = fl.create_federated_server_leaf_dataloader(
    server_train_images, server_train_labels, configs['runtime']['batch_size'], True)





2020-11-12 08:20:57,093 Select data from only 50 workers to be used...
2020-11-12 08:20:57,095 50
2020-11-12 08:20:57,096 Creating the server data from 50 selected users...
2020-11-12 08:20:57,128 Selected (1602, 28, 28) samples in total for the server from 50 users.
2020-11-12 08:20:57,130 Creating federated server dataloader...
2020-11-12 08:20:57,138 Scanning and sending data to server...


In [4]:
test_raw_data = utils.preprocess_leaf_data(
    utils.load_leaf_test(configs['data']['FEMNIST_PATH']),
    min_num_samples=0,only_digits=True)
server_test_images, server_test_labels = fl.create_server_femnist_dataset(
    test_raw_data, workers_idx, 100)

server_federated_test_dataloader = fl.create_federated_server_leaf_dataloader(
    server_test_images, server_test_labels, configs['runtime']['test_batch_size'], True)

2020-11-12 08:20:57,229 Loading test dataset from /home/ubuntu/data/leaf_non_iid/data/femnist/data
2020-11-12 08:20:57,231 Loading 1 out of 35 files...
2020-11-12 08:20:57,750 Loading 2 out of 35 files...
2020-11-12 08:20:58,228 Loading 3 out of 35 files...
2020-11-12 08:20:58,611 Loading 4 out of 35 files...
2020-11-12 08:20:59,185 Loading 5 out of 35 files...
2020-11-12 08:20:59,726 Loading 6 out of 35 files...
2020-11-12 08:20:59,966 Loading 7 out of 35 files...
2020-11-12 08:21:00,231 Loading 8 out of 35 files...
2020-11-12 08:21:00,490 Loading 9 out of 35 files...
2020-11-12 08:21:00,877 Loading 10 out of 35 files...
2020-11-12 08:21:01,127 Loading 11 out of 35 files...
2020-11-12 08:21:01,660 Loading 12 out of 35 files...
2020-11-12 08:21:02,093 Loading 13 out of 35 files...
2020-11-12 08:21:02,431 Loading 14 out of 35 files...
2020-11-12 08:21:02,649 Loading 15 out of 35 files...
2020-11-12 08:21:03,266 Loading 16 out of 35 files...
2020-11-12 08:21:03,523 Loading 17 out of 35 f

In [6]:
len(server_test_labels)

585